In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd 
import pandas_ta as ta # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, timedelta, time ,timezone #to specify the date time range for historical data
import time
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [4]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52584692
password = 'apD2lh!ZGwmr1P'
server = 'ICMarketsSC-Demo'
mt5.login(login,password,server)

True

In [5]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df

In [ ]:
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime.now() - timedelta(days=30)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# hour4 df
timeframe2 = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe2, start_datetime, end_datetime)
# daily df
timeframe3 = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe3, start_datetime, end_datetime)
ohlc_df.tail(1)

In [ ]:
def mystrategy():
    symbol = 'EURUSD'
    timeframe = mt5.TIMEFRAME_M1
    start_datetime = datetime.now() - timedelta(days=30)
    end_datetime = datetime.now()
    ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
    # hour4 df
    timeframe2 = mt5.TIMEFRAME_H4
    hourly_df = get_ohlc(symbol, timeframe2, start_datetime, end_datetime)
    # daily df
    timeframe3 = mt5.TIMEFRAME_D1
    daily_df = get_ohlc(symbol, timeframe3, start_datetime, end_datetime)
    #ohlc4
    daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
    hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
    #sma
    daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
    daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
    hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
    hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
    daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
    daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
    daily_df = daily_df.tail(1)
    hourly_df = hourly_df.tail(1)
    ohlc_df = ohlc_df.tail(1)
    def entring_stoploss_profit(row, hourly_df) :
        hour = row['time'].hour
        if row['sma_1'] > row['sma_14'] and row["sma_11"] > row['sma_144'] :#and(hour > 8):
            return "buy"
        elif row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_144'] :
            return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
    merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
    merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
    merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, hourly_df = hourly_df)

    # Tag first signal per day in original merged_df
    merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

    # Set signal only on the first row per day, others = None
    merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
    merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "tick_volume_x": "tick_volume",
    "spread_x" : "spread",
    "real_volume_x" : "real_volume",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close",
    "tick_volume": "daily_tick_volume",
    "spread" : "daily_spread",
    "real_volume" : "daily_real_volume"
    })
    tick = mt5.symbol_info_tick(symbol)
    lot = 0.10
    price = tick.ask
    buy_order_type = mt5.ORDER_TYPE_BUY
    sell_order_type = mt5.ORDER_TYPE_SELL
    buy_positions = mt5.positions_get(symbol=symbol)

    if hourly_df["sma_11"].iloc[-1] < hourly_df["sma_144"].iloc[-1]:
            buy_sl = ohlc_df["close"].iloc[-1]
    else:
         buy_sl = None
    if hourly_df["sma_11"].iloc[-1] > hourly_df["sma_144"].iloc[-1]:
            sell_sl = ohlc_df["close"].iloc[-1]
    else:
        sell_sl = None

    if merged_df["signal"].iloc[-1] == "buy" and not len(buy_positions) :
        request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": buy_order_type,
        "price": price,
        "sl": buy_sl,
        #"tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": f"buy executed {price}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
        result = mt5.order_send(request)

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print(f"buy trade placed successfully at {price} at {merged_df["time"].iloc[-1]}")
            print(f"SL: {buy_sl}")
        
    elif merged_df["signal"].iloc[-1] == "sell" and not len(buy_positions) :
        request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": sell_order_type,
        "price": tick.bid,
        "sl": sell_sl,
        #"tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "sell executed",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        print("✅ Trade Request:", request)
        if result is None:
            print("order_send failed: no result returned")
            print("Error code:", mt5.last_error())
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print(f"sell trade placed successfully at {price} at {merged_df["time"].iloc[-1]}")
            print(f"SL: {sell_sl}")
    else:
        print(f"No trade condition met → skipping at {merged_df["time"].iloc[-1]}")
            
    
    

In [ ]:
import schedule
schedule.every(30).seconds.do(mystrategy)

print("------> Strategy scheduler is running every 60 seconds...")

while True:
    schedule.run_pending()
    time.sleep(1)

------> Strategy scheduler is running every 60 seconds...


'while True:\n    schedule.run_pending()\n    time.sleep(5)'

In [ ]:
#ohlc4
daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
#sma
daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
daily_df = daily_df.tail(1)
hourly_df = hourly_df.tail(1)
ohlc_df = ohlc_df.tail(1)
daily_df


,time,open,high,low,close,tick_volume,spread,real_volume,ohlc,sma_1,sma_14,high_20,low_20
20,2025-10-20,1.1662,1.16757,1.16384,1.16409,51098,0,0,1.165425,1.165425,1.165676,1.17145,1.164673


In [ ]:
def entring_stoploss_profit(row, hourly_df) :
    hour = row['time'].hour
    if row['sma_1'] > row['sma_14'] and row["sma_11"] > row['sma_144'] :#and(hour > 8):
        return "buy"
    elif row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_144'] :
        return "sell"
# Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, hourly_df = hourly_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal','time']]

,signal,time
0,sell,2025-10-20 23:32:00


In [ ]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "tick_volume_x": "tick_volume",
    "spread_x" : "spread",
    "real_volume_x" : "real_volume",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close",
    "tick_volume": "daily_tick_volume",
    "spread" : "daily_spread",
    "real_volume" : "daily_real_volume"
})
print(merged_df.columns.tolist())
merged_df


['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume', 'hourly_open', 'hourly_high', 'hourly_low', 'hourly_close', 'tick_volume_y', 'spread_y', 'real_volume_y', 'ohlc_x', 'sma_11', 'sma_144', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'daily_tick_volume', 'daily_spread', 'daily_real_volume', 'ohlc_y', 'sma_1', 'sma_14', 'high_20', 'low_20', 'signal', 'is_first_signal']


,time,open,high,low,close,tick_volume,spread,real_volume,hourly_open,hourly_high,...,daily_tick_volume,daily_spread,daily_real_volume,ohlc_y,sma_1,sma_14,high_20,low_20,signal,is_first_signal
0,2025-10-20 23:32:00,1.16439,1.16441,1.16439,1.16441,3,0,0,1.16495,1.16529,...,51098,0,0,1.165425,1.165425,1.165676,1.17145,1.164673,sell,True


In [ ]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        #stop_loss_pips = (data['high'] - data["low_20"]) # example SL
        volume = 50000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        #stop_loss_pips = (data['high_20'] - data["low_20"]) # example SL
        volume = 50000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["sma_11"] < data["sma_144"] :
            orders.close_trade(trade) 
        elif trade['order_type'] == 'sell' and data["sma_11"] > data["sma_144"] :
            orders.close_trade(trade)


In [ ]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 50000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,50000,2025-10-20 23:32:00,1.16439,2025-10-20 23:32:00,1.16441,0,0,{},-1.0,-7.0,-8.0,-8.0,9992.0


In [ ]:
mm = bt.plot_balance()
display(mm)

In [ ]:
evaluate_backtest(bt.trades)

Daily Drawdown : -1.00
Max Drawdown (portfolio): -1.00
worst trades (trade MAE): [np.float64(-0.999999999995449)]
max_intrabar_drawdown: -1.00
biggest_profit: -1.0
balance_daily_drawdown: -1.0
balance_max_drawown: 0.0
avg_win: nan
avg_loss: -1.0
count_profit_trades: 0
count_loss_trades: 1
Win Rate1: 0.00%
rrr: nan


,order_type,profit
0,sell,-1.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown1


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown1
0,closed,EURUSD,sell,50000,2025-10-20 23:32:00,1.16439,2025-10-20 23:32:00,1.16441,0,0,...,-1.0,-7.0,-8.0,-8.0,9992.0,2025-10-20,2025-10-20 23:32:00,-1.0,-8.0,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,date,time,intrabar_drawdown,current_max,drawdown1,dayofweek,hourofday,month,year,drawdown
0,closed,EURUSD,sell,50000,2025-10-20 23:32:00,1.16439,2025-10-20 23:32:00,1.16441,0,0,...,2025-10-20,2025-10-20 23:32:00,-1.0,-8.0,0.0,0,23,2025-10,2025,0.0


max_drawdown 0.0
